In [6]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import pandas as pd
import os.path
import time

# создаю пустые списки, которые далее используются для создания финального датафрейма
download_dir = input('Введите директорию для загрузки файлов. Туда сохранятся файлы, необходимые для работы программы, и финальная таблица.')
options = wd.ChromeOptions()

profile = {"plugins.always_open_pdf_externally": True,
               "download.default_directory": download_dir , "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)

chrdr = input('Введите директорию, в которой хранится вебдрайвер')
driver = wd.Chrome(chrdr, options=options)

# скачиваю все необходимые файлы

pdf_url = 'https://www.heritage.org/index/download?version=821#'
driver.get(pdf_url)
driver.find_element_by_css_selector('#show-older-editions').click()
for i in [2,5,8]:
    driver.find_element_by_css_selector(f'#older-editions > a:nth-child({i})').click()

# циклом проверяю, закончилась ли загрузка последнего файла (за 2017 год)
# если не закончилась, код после while не выполняется
# если закончилась, создается список из всех скачанных датасетов 
# и закрывается окно браузера

while not os.path.exists(download_dir+f'\index2017_data.xls'):
    time.sleep(1)
if os.path.isfile(download_dir+f'\index2017_data.xls'):
    years = [2017,2018,2019]
    all_datasets = {year:pd.read_excel(download_dir+f'\index'+str(year)+'_data.xls') for year in years}
    driver.quit()
    
countries = []
index = []
years2 = []
for year, data in zip(years, all_datasets.values()):
    for cun, ind in zip(data['Country Name'], data[f'{year} Score']):
        countries.append(cun)
        index.append(ind)
        years2.append(year)
        
EFI = pd.DataFrame({'Year':years2, 'Index of Economic Freedom':index}, index=countries)